In [31]:
import json
from typing import Literal
import os
import pandas as pd
from unicodedata import normalize
import seaborn as sns

In [16]:
DATASET_ROOT = "../datasets_raw/styl_literacki/"

with open(f"{DATASET_ROOT}/dataset.json") as f:
    meta = json.load(f)

In [17]:
meta

{'train': ['2000_Milosz-Czeslaw_Koment-do-Listu.txt',
  '2007_Liberkowski-Ryszard_O-fakcie.txt',
  '1997_Skarga-Barbara_Tozsamosc-czy-roznica.txt',
  '2017_Zagajewski-Adam_Zimowa-podroz.txt',
  '2009_Liberkowski-Ryszard_Wszechswiat.txt',
  '1962_Herbert-Zbigniew_Arles.txt',
  '1957_Wyka-Kazimierz_Dwie-jesienie.txt',
  '1994_Matywiecki-Piotr_Deixis.txt',
  '1962_Jastrun-Mieczyslaw_Chmura.txt',
  '1970_Kepinski-Antoni_Biologiczny.txt',
  '2007_Zagajewski-Adam_Spiewal-o-jasnosci.txt',
  '1974_Milosz-Czeslaw_Rzeczywistosc.txt',
  '1989_Baranczak-Stanislaw_O-pisaniu-wierszy.txt',
  '2004_Tokarska-Bakir-Joanna_Topos-ruin.txt',
  '1972_Kijowski-Andrzej_Rewolucje.txt',
  '1982_Baranczak-Stanislaw_Kto-jest-dysydentem.txt',
  '1992_Sadurski-Wojciech_Panstwo-a-religia.txt',
  '1957_Stempowski-Jerzy_Nad-wodospadem.txt',
  '2018_Kaczmarek-Tomasz_Grand.txt',
  '1957_Stempowski-Jerzy_Klimat.txt',
  '2009_Liberkowski-Ryszard_Wyobrazniowa.txt',
  '1994_Kolakowski-Leszek_Po-co-uniwersytet.txt',
  '1972_

In [25]:
def normalize_json_strings(input_path, output_path=None):
    # If not specified, overwrite the input file
    if output_path is None:
        output_path = input_path

    # Read the original JSON data
    with open(input_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Function to recursively normalize all strings in the data
    def normalize_nested_data(data):
        if isinstance(data, dict):
            return {normalize('NFC', key): normalize_nested_data(value) for key, value in data.items()}
        elif isinstance(data, list):
            return [normalize_nested_data(element) for element in data]
        elif isinstance(data, str):
            return normalize('NFC', data)
        else:
            return data

    # Normalize strings in the JSON data
    normalized_data = normalize_nested_data(data)

    # Write the normalized data back to a new or the same JSON file
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(normalized_data, file, ensure_ascii=False, indent=4)

In [26]:
normalize_json_strings(os.path.join(DATASET_ROOT, "dataset.json"), os.path.join(DATASET_ROOT, "dataset_unicode_normalized.json"))

In [27]:
def texts_to_dataframe(meta: dict) -> pd.DataFrame:
    train_filenames = meta["train"]
    test_filenames = meta["test"]
    file_to_label: dict[str, Literal['naukowy', 'literacki']] = meta["file_to_label"]
    
    data_root = os.path.join(DATASET_ROOT, "dane")
    records: list[dict[str, any]] = []
    
    for filename in train_filenames:
        style_label = file_to_label[filename]
        records.append(
            file_to_record(data_root, filename, style_label, fold="train")
        )
        
    for filename in test_filenames:
        style_label = file_to_label[filename]
        records.append(
            file_to_record(data_root, filename, style_label, fold="test")
        )
        
    return pd.DataFrame.from_records(records)


def file_to_record(data_root: str, file_name: str, literary_style: Literal['naukowy', 'literacki']
                   , fold: Literal['train', 'test']) -> dict:
    # parse filename: YYYY_Some-Name_Some-Title.txt
    year, author, title = file_name.removesuffix(".txt")\
        .split("_")
    
    content_path = os.path.join(data_root, file_name)
    
    with open(content_path, "r") as f:
        content = f.read()
    
    return dict(
        title=title,
        author=author,
        year=year,
        text=content,
        style=literary_style,
        fold=fold
    )

In [28]:
with open(f"{DATASET_ROOT}/dataset_unicode_normalized.json") as f:
    meta = json.load(f)
    
meta

{'train': ['2000_Milosz-Czeslaw_Koment-do-Listu.txt',
  '2007_Liberkowski-Ryszard_O-fakcie.txt',
  '1997_Skarga-Barbara_Tozsamosc-czy-roznica.txt',
  '2017_Zagajewski-Adam_Zimowa-podroz.txt',
  '2009_Liberkowski-Ryszard_Wszechswiat.txt',
  '1962_Herbert-Zbigniew_Arles.txt',
  '1957_Wyka-Kazimierz_Dwie-jesienie.txt',
  '1994_Matywiecki-Piotr_Deixis.txt',
  '1962_Jastrun-Mieczyslaw_Chmura.txt',
  '1970_Kepinski-Antoni_Biologiczny.txt',
  '2007_Zagajewski-Adam_Spiewal-o-jasnosci.txt',
  '1974_Milosz-Czeslaw_Rzeczywistosc.txt',
  '1989_Baranczak-Stanislaw_O-pisaniu-wierszy.txt',
  '2004_Tokarska-Bakir-Joanna_Topos-ruin.txt',
  '1972_Kijowski-Andrzej_Rewolucje.txt',
  '1982_Baranczak-Stanislaw_Kto-jest-dysydentem.txt',
  '1992_Sadurski-Wojciech_Panstwo-a-religia.txt',
  '1957_Stempowski-Jerzy_Nad-wodospadem.txt',
  '2018_Kaczmarek-Tomasz_Grand.txt',
  '1957_Stempowski-Jerzy_Klimat.txt',
  '2009_Liberkowski-Ryszard_Wyobrazniowa.txt',
  '1994_Kolakowski-Leszek_Po-co-uniwersytet.txt',
  '1972_

In [29]:
df = texts_to_dataframe(meta)

In [30]:
df

,title,author,year,text,style,fold
0,Koment-do-Listu,Milosz-Czeslaw,2000,W 1936 roku Polska miała z jednej strony imper...,literacki,train
1,O-fakcie,Liberkowski-Ryszard,2007,"Często, choćby nieświadomie, porównujemy własn...",naukowy,train
2,Tozsamosc-czy-roznica,Skarga-Barbara,1997,Tożsamość jest jedną z najstarszych kategorii ...,naukowy,train
3,Zimowa-podroz,Zagajewski-Adam,2017,W drugim tomie rozmów z Osvaldo Ferrarim - kon...,literacki,train
4,Wszechswiat,Liberkowski-Ryszard,2009,"W obliczu mało precyzyjnego, gdyż potocznego p...",naukowy,train
...,...,...,...,...,...,...
616,Wertykalnosc,Kruszewski-Tomasz,2016,W czasach ponowoczesnych człowiek chce być mia...,naukowy,test
617,Wszystkie-komety,Maliszewski-Krzysztof,2013,Straty są jak ocean albo powietrze — oblegają ...,naukowy,test
618,Poszukiwanie,Maliszewski-Krzysztof,2015,"Czesław Miłosz, komentując własną słynną frazę...",naukowy,test
619,Freud-pozadanie,Bienczyk-Marek,1998,Czy sprawa to dla Freuda? W każdym razie nazwi...,literacki,test


In [34]:
df["text_length"] = df["text"].apply(lambda s: len(s))

In [36]:
df.sort_values(by="text_length", ascending=False)

,title,author,year,text,style,fold,text_length
534,Fantastyka1,Lem-Stanislaw,1970,Jeśli napisanie monografii wyczerpującej jakąś...,literacki,test,1536084
509,Wielokrok,Slawek-Tadeusz,2015,Ludwig Wittgenstein: „Pytanie: Jak idziesz prz...,literacki,test,349228
577,Fantastyka2,Lem-Stanislaw,1970,"Prawidłowością dynamiki cywilizacyjnej, której...",literacki,test,320805
118,Tozsamosc-ja,Skarga-Barbara,1997,Mówić dziś o tożsamości Ja może być poczytane ...,naukowy,train,233759
56,Os-egzystencji,Liberkowski-Ryszard,2011,Życie codzienne w najbardziej nawet prozaiczny...,naukowy,train,219451
...,...,...,...,...,...,...,...
86,Swiadomosc,Matywiecki-Piotr,1994,"Bycie świadomym jest uświadamiane refleksją, w...",literacki,train,1586
327,No-more,Kepinski-Antoni,1963,Dialog majora Eatherly z filozofem wiedeńskim ...,naukowy,train,1559
306,Wiezienie,Matywiecki-Piotr,1994,Czerniaków: Dziennik. 22 III 1942 r.: W areszc...,literacki,train,1431
23,Obrona-Diogenesa,Flaszen-Ludwik,1996,"Działaj, działaj, działaj! Czyż nie dość, że o...",literacki,train,1419


In [37]:
# NOTE: zawiera jakieś \n -> możliwe że do usunięcia

In [38]:
df.to_parquet(os.path.join(DATASET_ROOT, "styl_literacki.parquet"))

### Text splits

In [53]:
from langchain.text_splitter import SpacyTextSplitter

In [54]:
splitter = SpacyTextSplitter(pipeline="pl_core_news_lg")

In [55]:
long_text = df.query("title == 'Postscriptum' and author == 'Milosz-Czeslaw'")["text"].item()

In [56]:
chunks = splitter.split_text(long_text)

In [58]:
len(chunks)

8

In [59]:
[len(chunk) for chunk in chunks]

[3871, 3933, 3990, 3835, 3997, 3871, 3827, 3176]

In [60]:
import pandas as pd

In [62]:
df = pd.read_parquet("../data/LiteraryStyle/raw/train.parquet")

In [63]:
df

,chunked_text,label
0,W 1936 roku Polska miała z jednej strony imper...,0
1,"Często, choćby nieświadomie, porównujemy własn...",1
1,Takie ostre przeciwstawienie stanowisk dzisiaj...,1
1,"Kategorie naszego języka, kształtowane przez w...",1
1,Schopenhauera). Uniwersalność wszakże pozwoli ...,1
...,...,...
496,Doktryna społeczna wyłożona w Katechizmie... j...,1
496,Dekrety soboru trydenckiego w tej sprawie zost...,1
497,Roman Ingarden był bogiem filozofów. Gdy zaczą...,0
497,"Niestety, opowiadania byłych studentów Ingarde...",0
